<center><h1> Exploratory Data Analysis of car dataseet </h1></center>
***

In this notebook, I will explore and analyze my data, visuliaze connections between features and do some statistic manipulations to show main characteristics of data. First I will import data, that "page-scarper.py" got from [cars.kg](cars.kg) and saved in "cars_raw_data.csv". I called raw, because later I will clean the data: fill missing values, detect and maybe eliminate outliers, and maybe rescale data(standartization, normalization).

I will import pandas, which is a library for data analysis, and you can see first 5 rows and last five raws of the data.

In [1]:
import sys
sys.path

['',
 'C:\\Users\\AsusInside\\Anaconda3\\python36.zip',
 'C:\\Users\\AsusInside\\Anaconda3\\DLLs',
 'C:\\Users\\AsusInside\\Anaconda3\\lib',
 'C:\\Users\\AsusInside\\Anaconda3',
 'C:\\Users\\AsusInside\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\AsusInside\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\AsusInside\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\AsusInside\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\AsusInside\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\AsusInside\\.ipython']

In [1]:
import pandas as pd 
pd.set_option('float_format', '{:.3f}'.format)
from src.car_price_prediction.utils import dataset_manager

car_data = dataset_manager.get_raw_dataset()
car_data.head(5)

ModuleNotFoundError: No module named 'src'

In [ ]:
car_data.tail(5)

The very first thing I do, is check the shape of our dataframe.

In [ ]:
car_data.shape

Next we can see the quantity of missing values in each of the columns. We can see that some of the features have a lot of missing values, and some not so much. I will go three ways, first I will create a model with a dataset where all the NaN values were dropped, and second I will impute all the missing NaN values and the third I will try to make maximum unbiased dataset, managing the data as a field expert. All of the three datasets will have different level of bias, I will explore it in 'Choosing-Dataset' notebook.


In [ ]:
car_data.isnull().sum()

Next I would like to see detailed statistical summary of data. From the summary, we can see that mean of data and standard deviation have extreme values, and looking at max values at dataset, I can suggest that maximum values skewed the mean and std methods, later I will eliminate or impute those extreme values in dataset, also in next summary we can see the datatype of each column that we have.

In [ ]:
car_data.describe()

In [ ]:
car_data.info()

Next, I would want to see the correlation matrix between each of the continous(numeric) features.  
By looking at summary above, I now know that dataset has extreme values in the data, each one of continious variables contains extreme values, which we need to eliminate, or make it NaN and impute, because if we won't, it will make a huge affect on correlations and predictions.  
In the plot below we can see that correlation between continous variables is very low, it is due to the skewed data. There is no meaining at keeping analysis with this kind of data, at first I will eliminate extreme values just to see important relationships, I will not change dataset, the dataset will be changed and cleaned in "Data-Cleaning" notebook. I will clean data by "cleaning-outliers.py" script, by eliminating meaningless data, like: impossible mileage, capacity or engine power.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
%matplotlib inline

def plot_heatmap(data):
    sns.set(style = "white")
    corr = data.corr()
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    f, ax = plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
    
plot_heatmap(car_data)

In the dataset categorical features don't have extreme values, and continuous feature "Год выпуска" doesn't have extreme values too. I think it is due to the way people could post advertisements. Categorical features and year feature could be inputed via selection, and continous numbers could be of any range.

In [ ]:
car_data.isnull().sum()

Now I will download data cleaned from outliers and impossible values, and we can see that number of NaN values increased, I made some extreme values NaN, so later i could impute that data. After, I will drop some non-needed values.

In [ ]:
car_data = dataset_manager.get_cleaned_outliers_dataset()
car_data.isnull().sum()

Now we can continue our Exploritary Analysis. And right away we can see, that all statistic summary now makes full sense. And correlation plotting now is a little bit more reassuring.

In [ ]:
car_data.describe()

In [ ]:
plot_heatmap(car_data)

In [ ]:
car_data.corr()

Now it's time to see, which features of dataset are important and which are not so. I am going to use RandomForest Regressor module from scklearn library to analyze importance of features. Also, I wll use OneHotEncoder to make categorical features numeric.  
I will drop the NaN values for this example, because RandomForestRegressor doesn't take as an input data with missing values.
I will replace NaN values in urgency column, cause there is a lot of missing values there.  
And also I will drop power column, because of several reasons:
* Power column is dependent from capacity column on 87.5 per cent, we can see it above in correlation matrix
* Most of the people in KG doesn't care about the power, but do about capacity, that's why there are a lot of NaN values in power column, and a lot of impossible values
* Later in notebooks, we will see that power column actually deteriorate prediction accuracy

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from src.car_price_prediction.utils import df_utils

car_data.dropna(inplace=True)
car_data.drop(columns = ["Мощность"], inplace = True)

In [ ]:
X,y = df_utils.get_data_and_target(car_data)
X_dummies = pd.get_dummies(X)
forest = RandomForestRegressor()
feat_labels = X_dummies.columns[:-1]
forest.fit(X_dummies,y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
    
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), forest.feature_importances_), X_dummies.columns.values), reverse=True))

Now we will make use of dimensionality reduction algorithm which is called Principal Component Analysis(PCA). It will allow us to reduce the number of dimensions of a dataset. To visualize data I will reduce the number of dimensions to two. At first we will scale the whole dataset so it will have near to equal sizes. We will scale dataset via StandartScaler module of scikit learn module.From this plot we can see the distribution of data, and outliers as well.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

pca = PCA(n_components=1)
scaler = StandardScaler()
X_scaled = df_utils.scale_train(X)
X_pca = pd.get_dummies(X_scaled)
X_pca = pca.fit_transform(X_pca)
plt.scatter(X_pca,y,marker = "o",lw=0.1)

In [ ]:
car_data.rank().corr('spearman')

We finished the exlporitary analysis of our data. Next thing I will do is a cleaning the data.